# Test notebook to exercise the Lasair API

Important: change this to the correct import!

In [1]:
!/usr/bin/pip3 install lasair --upgrade
!/usr/bin/python3 --version
import sys

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Python 3.9.6


Change this to the API token and endpoint you want to test

In [2]:
lsst=True # if you change this restart kernel

if lsst:
    sys.path.append('API_lsst')
    import settings
    endpoint = "https://lasair-lsst.lsst.ac.uk/api"
    oid = 'diaObjectId'
    raname = 'ra'
    decname = 'decl'
else:
    sys.path.append('API_ztf')
    import settings
    endpoint = "https://lasair-ztf.lsst.ac.uk/api"
    oid = 'objectId'
    raname = 'ramean'
    decname = 'decmean'

Fetch your API TOKEN from your Lasair account, top right under "profile"

In [3]:
import json
from lasair import LasairError, lasair_client as lasair
import settings
L = lasair(settings.API_TOKEN, endpoint=endpoint)

/Users/rwillia5/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Cone search

In [4]:
racone  = 279.22
deccone = -47.91
radius = 60 # arcsec
try:
    result = L.cone(racone, deccone, radius, requestType='nearest')
    print(json.dumps(result, indent=2))
except:
    print('nothing found')

{
  "nearest": {
    "object": 3068394074097582956,
    "separation": 38.04260238794738
  }
}


## Database query

In [5]:
selected = 'objects.%s, objects.%s, objects.%s' % (oid, raname, decname)
tables = 'objects,sherlock_classifications'
conditions = 'classification="SN"'
results = L.query(selected, tables, conditions, limit = 10)

for row in results:
    objectId = row[oid]
    ra = row[raname]
    dec = row[decname]
    print(objectId, ra, dec)

# Just use this objectId in subsequent calls
objectId = str(objectId)

3068394045085582376 279.4857903368685 -47.84907173583371
3068394045085582397 279.4911834960133 -47.851086891736536
3068394045093970099 279.50723719217103 -47.959219936975515
3068394045093970122 279.5092098287064 -47.96506904230121
3068394045098164290 279.57432447327443 -47.898577765375954
3068394045098164346 279.59900590033993 -47.89783886044228
3068394045098164368 279.5981865459994 -47.90226079565548
3068394045098164397 279.60419470778953 -47.90672790398636
3068394045098164399 279.5485257345532 -47.93950914859985
3068394045098164648 279.5885176313158 -47.98057858881655


## Lightcurves

In [6]:
result = L.lightcurves([objectId])
print(json.dumps(result, indent=2))

[
  {
    "diaObjectId": "3068394045098164648",
    "diaSources": [
      {
        "diaSourceId": 3068394045098164648,
        "midpointMjdTai": 60487.05553157995,
        "ra": 279.58852422822304,
        "decl": -47.980596512862036,
        "band": "r",
        "psfFlux": 5206.4853515625,
        "psfFluxErr": 770.6403198242188
      },
      {
        "diaSourceId": 3068394047233065052,
        "midpointMjdTai": 60487.05646859384,
        "ra": 279.5885110344132,
        "decl": -47.98056066477067,
        "band": "r",
        "psfFlux": 10816.3046875,
        "psfFluxErr": 751.903564453125
      }
    ],
    "diaForcedSources": [
      {
        "midpointMjdTai": 60487.05646859384,
        "band": "r",
        "psfFlux": 10832.7724609375,
        "psfFluxErr": 779.3530883789062
      }
    ]
  }
]


## Sherlock Object

In [7]:
lite=True
result = L.sherlock_object(objectId, lite=lite)
print(json.dumps(result, indent=2))

{
  "classifications": {
    "3068394045098164648": [
      "SN",
      "The transient is possibly associated with <em>18382354-4758568</em>; a J=14.02 mag galaxy found in the 2MASS catalogue. It's located 6.77\" N, 23.12\" W from the galaxy centre."
    ]
  },
  "crossmatches": [
    {
      "Mag": 14.015,
      "MagErr": 0.111,
      "MagFilter": "J",
      "association_type": "SN",
      "catalogue_object_id": "18382354-4758568",
      "catalogue_object_type": "galaxy",
      "catalogue_table_name": "2MASS",
      "classificationReliability": 2,
      "decDeg": -47.98246,
      "direct_distance": null,
      "distance": null,
      "eastSeparationArcsec": "23.125",
      "major_axis_arcsec": null,
      "northSeparationArcsec": "-6.773",
      "photoZ": null,
      "photoZErr": null,
      "physical_separation_kpc": null,
      "raDeg": 279.598114,
      "separationArcsec": 24.096,
      "transient_object_id": "3068394045098164648",
      "z": null
    }
  ]
}


## Sherlock Position

In [8]:
result = L.sherlock_position(ra, dec, lite=lite)
print(json.dumps(result, indent=2))

{
  "classifications": {
    "query0": [
      "SN",
      "The transient is possibly associated with <em>18382354-4758568</em>; a J=14.02 mag galaxy found in the 2MASS catalogue. It's located 6.77\" N, 23.12\" W from the galaxy centre."
    ]
  },
  "crossmatches": [
    {
      "Mag": 14.015,
      "MagErr": 0.111,
      "MagFilter": "J",
      "association_type": "SN",
      "catalogue_object_id": "18382354-4758568",
      "catalogue_object_type": "galaxy",
      "catalogue_table_name": "2MASS",
      "classificationReliability": 2,
      "decDeg": -47.98246,
      "direct_distance": null,
      "distance": null,
      "eastSeparationArcsec": "23.125",
      "major_axis_arcsec": null,
      "northSeparationArcsec": "-6.773",
      "photoZ": null,
      "photoZErr": null,
      "physical_separation_kpc": null,
      "raDeg": 279.598114,
      "separationArcsec": 24.096,
      "transient_object_id": "query0",
      "z": null
    }
  ]
}


## All object data

In [9]:
result = L.objects(objectId)
print(json.dumps(result, indent=2))

[
  {
    "diaObjectId": 3068394045098164648,
    "objectData": {
      "nSources": 2,
      "ra": 279.5885176313158,
      "decl": -47.98057858881655,
      "mjdmin": 60487.05553157995,
      "mjdmax": 60487.05646859384,
      "rasex": "18:38:21.244",
      "decsex": "-47:58:50.083",
      "ec_lon": 277.0539163377847,
      "ec_lat": -24.77580615578947,
      "now_mjd": "60571.34",
      "mjdmin_ago": 84.28506045615359,
      "mjdmax_ago": 84.28412344226672,
      "discMjd": 60487.05553157995,
      "discUtc": "2024-06-26 01:19:57",
      "discMag": "5206.49\u00b1770.64",
      "discFilter": "r",
      "latestMjd": 60487.05646859384,
      "latestUtc": "2024-06-26 01:21:18",
      "latestMag": "10816.30\u00b1751.90",
      "latestFilter": "r",
      "peakMjd": 60487.05553157995,
      "peakUtc": "2024-06-26 01:19:57",
      "peakMag": "5206.49\u00b1770.64",
      "peakFilter": "r"
    },
    "diaSources": [
      {
        "diaSourceId": 3068394047233065052,
        "midpointMjdTai": 